In [2]:
import os

from cnn_finetune import make_model
import pandas as pd
from skimage import io, transform
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms, utils

/Users/kyohei.watarai/.pyenv/versions/3.7.0/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
model = make_model('vgg16', num_classes=2, pretrained=True, input_size=(128, 128))

In [4]:
device = torch.device('cpu')
model = model.to(device)

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=model.original_model_info.mean,
        std=model.original_model_info.std),
])

In [6]:
class MyDataSet(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.data_df = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return self.data_df.shape[0]
    
    def __getitem__(self, idx):
        image_name = os.path.join(self.root_dir, self.data_df.iloc[idx, 0])
        image = io.imread(image_name)
        label = self.data_df.iloc[idx, 1]
        return self.transform(image), int(label)

In [7]:
train_set = MyDataSet('../images/train_data.csv', '../images/resize/data/', transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10, shuffle=True)

In [8]:
test_set = MyDataSet('../images/test_data.csv', '../images/resize/data/', transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=True)

https://github.com/creafz/pytorch-cnn-finetune/blob/master/examples/cifar10.py

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [10]:
def train(epoch):
    total_loss = 0
    total_size = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), total_loss / total_size))

In [11]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 6):
    train(epoch)
    test()

Train Epoch: 1 [0/173 (0%)]	Average loss: 0.078974

Test set: Average loss: 0.0809, Accuracy: 43/76 (57%)

Train Epoch: 2 [0/173 (0%)]	Average loss: 0.056893

Test set: Average loss: 0.0438, Accuracy: 62/76 (82%)

Train Epoch: 3 [0/173 (0%)]	Average loss: 0.028583

Test set: Average loss: 0.0786, Accuracy: 45/76 (59%)

Train Epoch: 4 [0/173 (0%)]	Average loss: 0.048614

Test set: Average loss: nan, Accuracy: 31/76 (41%)

Train Epoch: 5 [0/173 (0%)]	Average loss: nan
